In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [2]:
spark = SparkSession.builder\
    .config("spark.warehouse.dir","file:///c:/tmp")\
    .appName("SparkSQL").getOrCreate()

In [5]:
sc = spark.sparkContext
data =sc.textFile('peoplename.csv')
data.take(2)

['first_name ,last_name,gender,age', 'Ededd,Shankj,F,43']

In [6]:
df =spark.read.format("csv").option("header","true").load("peoplename.csv")

In [7]:
df.printSchema()

root
 |-- first_name : string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)



In [10]:
df = df.withColumn("age",df.age.cast("int"))
df.printSchema()
#数据类型转换

root
 |-- first_name : string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)



In [11]:
last_name = df.groupBy("last_name")

In [15]:
average =df.groupBy("last_name").agg({"age":"avg"})

In [16]:
#后期如果需要使用，cache后会更快
average.cache()

DataFrame[last_name: string, avg(age): double]

In [17]:
average.show()

+---------+--------+
|last_name|avg(age)|
+---------+--------+
|     Sjdh|    44.0|
|   Shankj|    43.0|
|      Lii|    43.0|
|       Wu|    23.0|
+---------+--------+



In [18]:
average.orderBy("avg(age)").show()

+---------+--------+
|last_name|avg(age)|
+---------+--------+
|       Wu|    23.0|
|   Shankj|    43.0|
|      Lii|    43.0|
|     Sjdh|    44.0|
+---------+--------+



In [19]:
#重命名
average =average.withColumnRenamed("avg(age)","average")

In [21]:
average.sort(average.average.desc()).show()
average.sort("average",ascending = False).show()

+---------+-------+
|last_name|average|
+---------+-------+
|     Sjdh|   44.0|
|   Shankj|   43.0|
|      Lii|   43.0|
|       Wu|   23.0|
+---------+-------+

+---------+-------+
|last_name|average|
+---------+-------+
|     Sjdh|   44.0|
|   Shankj|   43.0|
|      Lii|   43.0|
|       Wu|   23.0|
+---------+-------+



In [22]:
#排序：orderby sort 或者说输入列名的方式来改变
import pandas as pd
average.show()

+---------+-------+
|last_name|average|
+---------+-------+
|     Sjdh|   44.0|
|   Shankj|   43.0|
|      Lii|   43.0|
|       Wu|   23.0|
+---------+-------+



In [23]:
pandas_df =average.toPandas()
pandas_df.head()

,last_name,average
0,Sjdh,44.0
1,Shankj,43.0
2,Lii,43.0
3,Wu,23.0
